In [1]:
import numpy as np
import matplotlib.pyplot as plt

# Backpropagation

Suppose that we have data $(\mathbf{x}_i,y_i)_{1\le i\le n}$ where $\mathbf{x}\in\mathbb{R}^d$ and $y_i\in \mathbb{R}$ and we want do regression by finding a good function $f:\mathbb{R}^d\to\mathbb{R}$ that fits this data well:
$$
\sum_{i=1}^n \ell(f(\mathbf{x}_i),my_i)\text{ is small},
$$
where $C$ is some cost function.

We have seen two main ways to do this but both involve parametrizing the collection of functions $f$ so that they are really functions of some parameter $\boldsymbol{\theta}\in \mathbb{R}^p$ so that we really want to minimize
$$
\arg \min_{\boldsymbol{\theta}}\sum_{i=1}^n \ell\left(f(\mathbf{x}_i,\boldsymbol{\theta}),y_i\right).
$$

The examples that we've looked at have had comuptable by-hand (although perhaps ugly) expressions for their gradients and Hessians where we've been able to use convex analysis, and in particular gradient descent, to minimize the cost. 

What happens when things get more complicated?

Many important current-day models for regression and classification use multi-layer progressive functions. We'll stick with a classification problem here.

Our data is $(\mathbf{x}, y)$ has $\mathbf{x}\in \mathbb{R}^d$ and $y = \{label_1,label_2,\dotsm, _label_K\}$ is assigned one of $K$ many labels. In practice it is often easier to redefine the discrete options for $y$ into a continuous variable by saying that label $i\in\{1,\dotsm,K\}$ is actually the standard basis vector in $\mathbf{R}^K$. This transforms our data $y\in \{label_1,\dotsm,label_K\}$ to $\mathbf{y} \in \Delta^K := \{(p_1,\dotsm,p_K): \sum_{i=1}^K p_i = 1, p_i\ge 0\}.$

What this means is that we have start with data $\mathbf{z}_1 = \mathbf{x}\in \mathbb{R}^d$. This is our input for *layer 1*. We write $n_1 = d$.

We then apply a function $\mathbf{g}_1:\mathbb{R}^{n_1+ r_1} \to \mathbb{R}^{n_2}$ which takes our initial data $\mathbf{z}_1\in \mathbb{R}^{n_1}$ and some parameter vecotr $\mathbf{w}_1\in \mathbb{R}^{r_1}$ and gives us new data in $\mathbb{R}^{n_2}$. Formally
$$
\mathbf{z}_2 = \mathbf{g}_1 (\mathbf{z}_1, \mathbf{w}_1).
$$

We then take this transformed data and transform it again and again. We have functions
$$
\mathbf{g}_i:\mathbb{R}^{n_i+r_i}\to \mathbb{R}^{n_{i+1}},\qquad i = 1,2,\dotsm, L
$$
that takes an input $\mathbf{z}_i\in \mathbb{R}^{n_i}$ and some parameters $\mathbf{w}_i\in \mathbb{R}^{r_i}$ and gives us an output $\mathbf{z}_{i+1}\in \mathbb{R}^{n_{i+1}}$ that we feed into the next layer. 

At the end we have some function
$$
\mathbf{h}:\mathbb{R}^{d+ r_1+r_2+\dotsm+r_L}\to \mathbb{R}^{n_{L+1}},\qquad (n_{L+1} = K)
$$
defined by
$$
\mathbf{h}(\mathbf{x}, \overline{\mathbf{w}}) = \mathbf{g}_L\Bigg(\mathbf{g}_{L-1}\bigg(\dotsm, \mathbf{g}_2\Big(\mathbf{g}_1(\mathbf{x},\mathbf{w}_1), \mathbf{w}_2\Big),\dotsm),\mathbf{w}_{L-1}\bigg) ,\mathbf{w}_L  \Bigg).
$$

Finally, we want to minimize a cost $\ell( \mathbf{h}(\mathbf{x},\overline{\mathbf{w}}),\mathbf{y})$ or the empirical loss
$$
\frac{1}{n}\sum_{i=1}^n \ell(\mathbf{h}(\mathbf{x}_i,\overline{\mathbf{w}}),\mathbf{y}_i)
$$

**BUT HOW ON EARTH DO WE COMPUTE THE OPTIMAL PARAMETER!!!?!?!??!!?**

## Derivatives of vector valued functions

We first start with how to differentiate vector valued functions: $\mathbf{f}:\mathbb{R}^d\to \mathbb{R}^m$ is made up of $m$ many coordinate functions
$$
\mathbf{f}(\mathbf{x}) = \begin{bmatrix}
f_1(\mathbf{x})\\
f_2(\mathbf{x})\\
\vdots\\
f_m (\mathbf{x})
\end{bmatrix}
$$

We know that when we have just a single function $f:\mathbb{R}\to \mathbb{R}$ then 
$$
f(x+h) \approx f(x) + f'(x) h
$$
and
when we have $f:\mathbb{R}^d\to \mathbb{R}$ then
$$
f(\mathbf{x}+ h\mathbf{v}) \approx f(\mathbf{x}) + \frac{\partial f}{\partial \mathbf{v}}(\mathbf{x})h = f(\mathbf{x}) + \nabla f(\mathbf{x})^T \mathbf{v} h
$$
or using solely vectors
$$
f(\mathbf{x}+\mathbf{h}) \approx f(\mathbf{x}) + \nabla f(\mathbf{x})^T \mathbf{h}.
$$ 
**Both give linear approximations.** 

We extend this to vector valued functions. 
$$
\mathbf{f}(\mathbf{x}+\mathbf{h}) \approx \mathbf{f}(\mathbf{x}) + T( \mathbf{h})
$$
where $T$ is a linear approximation that sends the vector $\mathbf{h}$ to a new vector.

First we've added $\mathbf{h}$ to $\mathbf{x}\in \mathbb{R}^d$ and we need that to make sense! That means we need $T$ to take in vectors in $\mathbf{R}^{d}$. Also we've added $T(\mathbf{h})$ to $\mathbf{f}(\mathbf{x})$ which lives in $\mathbb{R}^m$ and we need that addition to make sense which forces $T(\mathbf{h})\in \mathbb{R}^m$.  

So the derivative of $\mathbf{f}$ at $\mathbf{x}$ ought to be a linear function which takes a vector in $\mathbb{R}^d$ and spits out a vector in $\mathbb{R}^m$. That means $T(\mathbf{h}) = A\mathbf{h}$ where $A\in \mathbb{R}^{m\times d}.$

Now let's make a definition and state a lemma.

**Definition:** We say that a function $\mathbf{f}:\mathbb{R}^d\to \mathbf{R}^m$ is **differentiable** at $\mathbf{x}$ if there is a matrix $A\in \mathbb{R}^{m\times d}$ such that
$$
\mathbf{f}(\mathbf{x}+\mathbf{h}) =\mathbf{ f}(\mathbf{x}) + A\mathbf{h} + \mathbf{r}(\mathbf{h})
$$
for some remainder term $\mathbf{r}$ such that
$$
\lim_{\mathbf{h}\to \mathbf{0}} \frac{\|\mathbf{r}(\mathbf{h})\|}{\|\mathbf{h}\|} = 0.
$$
(Both the matrix $A$ and the function $\mathbf{r}$ can depend on what $\mathbf{x}$ we look at.)

We call the (abstractly defined) matrix $A$ above the **differential** of $\mathbf{f}$ at $\mathbf{x}$.

**Theorem/Definition** Suppose that $\mathbf{f}:\mathbb{R}^d\to \mathbb{R}^m$ is a function and $\mathbf{x}_0\in \mathbb{R}^d$. Suppose that $\frac{\partial f_j}{\partial x_i}(\mathbf{x}_0)$ exists for all $i = 1,\dots,d$ and $j = 1,\dots, m$ and the partial derivatives are continuous at $\mathbf{x}_0$. Then the differential $A$ exists, is unique, and is equal to the **Jacobian** of $\mathbf{f}$ at $\mathbf{x}_0$ defined by
$$
J_{\mathbf{f}} (\mathbf{x}_0) = \begin{bmatrix}\displaystyle
\frac{\partial f_1}{\partial x_1}(\mathbf{x_0})&\displaystyle\frac{\partial f_1}{\partial x_2}(\mathbf{x_0})&\dotsm&\displaystyle\frac{\partial f_1}{\partial x_d}(\mathbf{x_0})\\
\displaystyle\frac{\partial f_2}{\partial x_1}(\mathbf{x_0})&\displaystyle\frac{\partial f_2}{\partial x_2}(\mathbf{x_0})&\dotsm&\displaystyle\frac{\partial f_2}{\partial x_d}(\mathbf{x_0})\\
\vdots&\vdots&&\vdots\\
\displaystyle\frac{\partial f_m}{\partial x_1}(\mathbf{x_0})&\displaystyle\frac{\partial f_m}{\partial x_2}(\mathbf{x_0})&\displaystyle\dotsm&\frac{\partial f_m}{\partial x_d}(\mathbf{x_0})
\end{bmatrix}.
$$
Note
$$
J_{\mathbf{f}}(\mathbf{x}_0) = \begin{bmatrix}
\displaystyle \nabla f_1(\mathbf{x}_0)^T\\
\displaystyle \nabla f_2(\mathbf{x}_0)^T\\
\dotsm\\
\displaystyle \nabla f_d(\mathbf{x}_0)^T\\
\end{bmatrix}.
$$

**Example** If $f:\mathbb{R}^d\to \mathbf{R}$ then its Jacobian is 
$$
J_f(\mathbf{x}) = \nabla f(\mathbf{x})^T.
$$

**Example** If $\mathbf{f}:\mathbb{R}^d\to \mathbf{R}^m$ is the affine map
$$
\mathbf{f}(\mathbf{x}) = A\mathbf{x}+\mathbf{b}
$$
then its Jacobian is 
$$
J_{\mathbf{f}}(\mathbf{x}) = A.
$$ Why? The matrix $A$ is the differential:
$$
\mathbf{f}(\mathbf{x}+\mathbf{h}) = A(\mathbf{x}+\mathbf{h}) + \mathbf{b} = A\mathbf{x}+\mathbf{b} + A\mathbf{h} = f(\mathbf{x})+A\mathbf{h} + \mathbf{0}
$$
where the zero at the end is the remainder.

Given two functions $f:X\to Y$ and $g:Y\to Z$ we write $g\circ f: X\to Z$ for $g(f(x))$. This is the **composition** of $f$ and $g$ and we say that $g$ is composed with $f$.

**Theorem (Chain Rule):** Let $\mathbf{f}:\mathbb{R}^d\to \mathbb{R}^m$ and $\mathbf{g}:\mathbb{R}^m\to \mathbb{R}^p$. Suppose that $\mathbf{f}$ is continuously differentiable at $\mathbf{x}_0$ and $\mathbf{g}$ is continously differentiable at $\mathbf{f}(\mathbf{x}_0)$. Then $\mathbf{g}\circ\mathbf{f}$ is continuously differentiable at $\mathbf{x}_0$ and its Jacobian 
$$
J_{\mathbf{g}\circ \mathbf{f}} (\mathbf{x}_0) =  J_{\mathbf{g}}(\mathbf{f}(\mathbf{x}_0)) J_{\mathbf{f}}(\mathbf{x_0}).
$$

*Proof:* By the theorem above, the Jacobian is the differential. Therefore we compute the differential. We write $A$ for the differential of $\mathbf{f}$ and $B$ for the differential of $\mathbf{g}$.
\begin{align*}
\mathbf{g}\left(\mathbf{f}(\mathbf{x}+\mathbf{h}) \right) &=  \mathbf{g}\Big( \mathbf{f}(\mathbf{x}_0)+ A \mathbf{h} + \mathbf{r}_1(\mathbf{h})\Big)\\
&= \mathbf{g}(\mathbf{f}(\mathbf{x}_0)) + B (A \mathbf{h} + \mathbf{r}_1(\mathbf{h})) + \mathbf{r}_2(A\mathbf{h}+\mathbf{r}_1(\mathbf{h}))\\
&= \mathbf{g}\circ \mathbf{f}(\mathbf{x}_0) + B A \mathbf{h} +\mathbf{r}_3(\mathbf{h}),\\
\mathbf{r}_3(\mathbf{h})&=A\mathbf{r}_1(\mathbf{h}) + \mathbf{r}_2(A\mathbf{h}+\mathbf{r}_1(\mathbf{h})).
\end{align*}
It is a little tedious to verify, but remainder satisfies
$$
\lim_{\mathbf{h}\to\mathbf{0}} \frac{\|\mathbf{r}_3(\mathbf{h})\|}{\|\mathbf{h}\|} = 0.
$$ (You should try to do this on your own!)

Since $BA =  J_{\mathbf{g}}(\mathbf{f}(\mathbf{x}_0)) J_{\mathbf{f}}(\mathbf{x_0})$ is the differential of $\mathbf{g}\circ\mathbf{f}$, the Jacobain statement follows. $\square$



## Back to backpropagation

Recall that we have a 
$$\mathbf{h}(\mathbf{x}, \overline{\mathbf{w}}) = \mathbf{g}_L\Bigg(\mathbf{g}_{L-1}\bigg(\dotsm, \mathbf{g}_2\Big(\mathbf{g}_1(\mathbf{x},\mathbf{w}_1), \mathbf{w}_2\Big),\dotsm),\mathbf{w}_{L-1}\bigg) ,\mathbf{w}_L  \Bigg)$$
and we want to minimize 
$$
\ell( \mathbf{h}(\mathbf{x},\overline{\mathbf{w}}),\mathbf{y}).$$


First since we are treating $\mathbf{x}$ and $\mathbf{y}$ as data that we are given, we never differentiate with respect to them. So we'll treat them as constants and ignore them in our notation of $\mathbf{h}$ and $\ell$.

$$\mathbf{h}(\overline{\mathbf{w}}) = \mathbf{g}_L\Bigg(\mathbf{g}_{L-1}\bigg(\dotsm, \mathbf{g}_2\Big(\mathbf{g}_1(\mathbf{x},\mathbf{w}_1), \mathbf{w}_2\Big),\dotsm),\mathbf{w}_{L-1}\bigg) ,\mathbf{w}_L  \Bigg)$$
and we want to minimize a loss $\ell(\mathbf{h}(\overline{\mathbf{w}}))$. 

We write
$$
f(\overline{\mathbf{w}}) = \ell(\mathbf{h}(\overline{\mathbf{w}}))
$$

Here we think of 
$$
\overline{\mathbf{w}} \in \mathbb{R}^{r_1+r_2+\dotsm+r_L}
$$
is a vector whose first $r_1$ coordinates are $\mathbf{w}_1$, whose second $r_2$ many coordinates are $\mathbf{w}_2$ and so on.

### Towards some compositions
Unfortunately the function $\mathbf{h}$ does not look like a composition right now. So how do we make this look like compositions? The idea is to view
$$
\mathbf{g}_i(\text{Layer }i\text{ input}) = \text{Layer }i\text{ output}
$$
and
$$
\text{Layer }i \text{ input} = \left(\text{Layer }i-1 \text{ output}, \mathbf{w}_i\right)
$$

The input to layer $i$ is some vector $$\mathcal{I}_i(\mathbf{w}_1,\dotsm,\mathbf{w}_{i-1},\mathbf{w}_i) = \mathcal{I}_i(\overline{\mathbf{w}}^{i}).$$ And the output at layer $i$ is some vector
$$
\mathcal{O}_i(\overline{\mathbf{w}}^i) = \mathbf{g}_i(\mathcal{I}_i(\overline{\mathbf{w}}^i)).
$$ Now
$$
\mathcal{I}_i(\overline{\mathbf{w}}^i ) = \Big(\mathcal{O}_{i-1}(\overline{\mathbf{w}}^{i-1}), \mathbf{w}_i\Big).
$$

### Jacobians of $\mathcal{I}$ 
So the Jacobian of the input has the very nice blockstructure
$$
J_{\mathcal{I}_i}(\overline{\mathbf{w}}^i) = \begin{bmatrix}\displaystyle
J_{\mathcal{O}_{i-1}}(\overline{\mathbf{w}}^{i-1}) & \displaystyle0_{n_{i}\times r_i}\\
\displaystyle0_{r_i\times n_i} &\displaystyle I_{r_i\times r_i}
\end{bmatrix}
$$
and since the input to the first layer is $\mathcal{I}_1 (\mathbf{w}_1) = \begin{bmatrix} \mathbf{x} \\\mathbf{w}_1\end{bmatrix}$ its Jacobian is
$$
J_{\mathcal{I}_1}(\mathbf{w}_1) = \begin{bmatrix}
0_{d\times r_1}\\
I_{r_1\times r_1}
\end{bmatrix}.
$$

### Jacobians of $\mathbf{g}$

Let's look at what the Jacobian of a function $\mathbf{g}(\mathbf{z},\mathbf{w})$ looks like:
$$
J_{\mathbf{g}}(\mathbf{z}_0,\mathbf{w}_0) = \begin{bmatrix}
\displaystyle \frac{\partial g_{1}}{\partial z_1}(\mathbf{z}_0,\mathbf{w}_0)&\dotsm &\displaystyle \frac{\partial g_{1}}{\partial z_n}(\mathbf{z}_0,\mathbf{w}_0)& \displaystyle \frac{\partial g_{1}}{\partial w_1}(\mathbf{z}_0,\mathbf{w}_0)&\dotsm&\displaystyle \frac{\partial g_{1}}{\partial w_r}(\mathbf{z}_0,\mathbf{w}_0)\\
\displaystyle \frac{\partial g_{2}}{\partial z_1}(\mathbf{z}_0,\mathbf{w}_0)&\dotsm &\displaystyle \frac{\partial g_{2}}{\partial z_n}(\mathbf{z}_0,\mathbf{w}_0)& \displaystyle \frac{\partial g_{2}}{\partial w_1}(\mathbf{z}_0,\mathbf{w}_0)&\dotsm&\displaystyle \frac{\partial g_{1}}{\partial w_r}(\mathbf{z}_0,\mathbf{w}_0)\\
\vdots&&\vdots&\vdots&&\vdots\\
\displaystyle \frac{\partial g_{m}}{\partial z_1}(\mathbf{z}_0,\mathbf{w}_0)&\dotsm &\displaystyle \frac{\partial g_{1}}{\partial z_n}(\mathbf{z}_0,\mathbf{w}_0)& \displaystyle \frac{\partial g_{m}}{\partial w_1}(\mathbf{z}_0,\mathbf{w}_0)&\dotsm&\displaystyle \frac{\partial g_{m}}{\partial w_r}(\mathbf{z}_0,\mathbf{w}_0)\\
\end{bmatrix}
$$
so we can write this in a "simple" block form as 
$$
J_{\mathbf{g}}(\mathbf{z},\mathbf{w}) = \begin{bmatrix} A & B\end{bmatrix}
$$ where the $A$ term carries all the information about the partial derivatives of ${z}$'s and $B$ carries all the information about the partial derivatives of $w$'s.

Recall that the input to layer $i$ is $$\mathcal{I}_i(\overline{\mathbf{w}}^i) = (\mathcal{O}_{i-1} (\overline{\mathbf{w}}^{i-1}),\mathbf{w}_i) = (\mathbf{z}_i, \mathbf{w}_i).$$

So we write 
$$
J_{\mathbf{g}_i}( \mathbf{z}_i, \mathbf{w}_i) = \begin{bmatrix}
A_i & B_i
\end{bmatrix}
$$ where the matrix $A\in \mathbb{R}^{n_{i+1}\times n_{i}}$ and $B\in \mathbb{R}^{n_{i+1}\times r_i}$.

### Jacobians of Early Layers

What is the Jacobian of the first layer? Recall that $A_1\in \mathbb{R}^{n_2\times n_1} = \mathbb{R}^{n_2\times d}$ and $B_1 \in \mathbb{R}^{n_2\times r_1}.$
$$
J_{\mathcal{O}_1}(\mathbf{w}_1) = J_{\mathbf{g}_1}(\mathbf{x},\mathbf{w}_1) J_{\mathcal{I}_1}(\mathbf{x},\mathbf{w}_1) = \begin{bmatrix} A_1 & B_1 \end{bmatrix} \begin{bmatrix} 0_{d\times r_1}\\I_{r_1\times r_1} \end{bmatrix} = A_1 0_{d\times r_1} + B_1\times I_{r_1\times r_1} = B_1.
$$

What's the Jacobian of the output of the second layer?
\begin{align*}
J_{\mathcal{O}_2}(\overline{\mathbf{w}}^2) &= J_{\mathbf{g}_2}(\mathcal{I}_2(\overline{\mathbf{w}}^2)) J_{\mathcal{I}_2}(\overline{\mathbf{w}}^2)\\
&=J_{\mathbf{g}_2}(\mathbf{z}_2,\mathbf{w}_2) J_{\mathcal{I}_2}(\overline{\mathbf{w}}^2)\\
&=\begin{bmatrix} A_2 & B_2 \end{bmatrix} \begin{bmatrix} 
J_{\mathcal{O}_1}(\overline{\mathbf{w}}^1) & 0\\
0 & I_{r_2\times r_2}
\end{bmatrix}\\
&=\begin{bmatrix}
A_2 J_{\mathcal{O}_1}(\overline{\mathbf{w}}^1) + B_2 0  & A_10+B_2 I_{r_2\times r_2} \\
\end{bmatrix}\\
&= \begin{bmatrix} A_2 J_{\mathcal{O}_1}(\overline{\mathbf{w}}^1) & B_2\end{bmatrix}.
\end{align*}

What about the general step?
\begin{align*}
J_{\mathcal{O}_i}(\overline{\mathbf{w}}^i) &= J_{\mathbf{g}_i}(\mathcal{I}_i(\overline{\mathbf{w}}^i)) J_{\mathcal{I}_i}(\overline{\mathbf{w}}^i)\\
&=J_{\mathbf{g}_i}(\mathbf{z}_i,\mathbf{w}_i) J_{\mathcal{I}_i}(\overline{\mathbf{w}}^i)\\
&=\begin{bmatrix} A_i & B_i \end{bmatrix} \begin{bmatrix} 
J_{\mathcal{O}_{i-1}}(\overline{\mathbf{w}}^{i-1}) & 0\\
0 & I_{r_i\times r_i}
\end{bmatrix}\\
&=\begin{bmatrix}
A_i J_{\mathcal{O}_{i-1}}(\overline{\mathbf{w}}^{i-1}) + B_i 0  & A_i0+B_i I_{r_i\times r_i} \\
\end{bmatrix}\\
&= \begin{bmatrix} A_i J_{\mathcal{O}_{i-1}}(\overline{\mathbf{w}}^{i-1}) & B_i\end{bmatrix}.
\end{align*}

### Gradient of the Cost:

What is the gradient of the cost $f(\overline{\mathbf{w}})$? 

$$
J_f(\overline{\mathbf{w}}) = \ell(\mathbf{h}(\overline{\mathbf{w}})) = J_{\ell}(\mathbf{h}(\overline{\mathbf{w}})) J_{\mathbf{h}}(\overline{\mathbf{w}}) = \nabla \ell\big(\mathcal{O}_L(\overline{\mathbf{w}}^L)\big)^T J_{\mathcal{O}_{L}}(\overline{\mathbf{w}}^L).
$$
But since $\nabla f = (J_f)^T$, we have

$$
\nabla f(\overline{\mathbf{w}}) = J_{\mathcal{O}_{L}}(\overline{\mathbf{w}}^L)^T \nabla \ell\big(\mathcal{O}_L(\overline{\mathbf{w}}^L)\big).
$$

### Putting things together:

We now know all the formulas to figure out what the gradient of the cost function actually is!

Note that $$J_{\mathcal{O}_i}(\overline{\mathbf{w}}^i)^T =  \begin{bmatrix} A_i J_{\mathcal{O}_{i-1}}(\overline{\mathbf{w}}^{i-1}) & B_i\end{bmatrix}^T =  
\begin{bmatrix}
J_{\mathcal{O}_{i-1}}(\overline{\mathbf{w}}^{i-1})^T A_i^T\\
B_i^T
\end{bmatrix}
\qquad i = 2,\dotsm, L$$
and
$$
J_{\mathcal{O}_1}(\overline{\mathbf{w}}^1)^T = B_1^T.$$

So
\begin{align*}
\nabla f(\mathbf{x})&= J_{\mathcal{O}_{L}}(\overline{\mathbf{w}}^L)^T \nabla \ell\big(\mathcal{O}_L(\overline{\mathbf{w}}^L)\big)\\
&= \begin{bmatrix}
J_{\mathcal{O}_{L-1}}(\overline{\mathbf{w}}^{L-1})^T A_L^T\\
B_L^T
\end{bmatrix} \nabla \ell\big(\mathcal{O}_L(\overline{\mathbf{w}}^L)\big)\\
&=\begin{bmatrix}
J_{\mathcal{O}_{L-1}}(\overline{\mathbf{w}}^{L-1})^T A_L^T\nabla \ell\big(\mathcal{O}_L(\overline{\mathbf{w}}^L)\big)\\
B_L^T\nabla \ell\big(\mathcal{O}_L(\overline{\mathbf{w}}^L)\big)
\end{bmatrix} \\
&=\begin{bmatrix}
\displaystyle \begin{bmatrix}
J_{\mathcal{O}_{L-2}}(\overline{\mathbf{w}}^{L-2})^T A_{L-1}^T\\
B_{L-1}^T
\end{bmatrix}A_{L}^T\nabla \ell\big(\mathcal{O}_L(\overline{\mathbf{w}}^L)\big)\\
B_L^T\nabla \ell\big(\mathcal{O}_L(\overline{\mathbf{w}}^L)\big)
\end{bmatrix}\\
&=
\begin{bmatrix}
\displaystyle 
J_{\mathcal{O}_{L-2}}(\overline{\mathbf{w}}^{L-2})^T A_{L-1}^TA_{L}^T\nabla \ell\big(\mathcal{O}_L(\overline{\mathbf{w}}^L)\big)\\
\displaystyle B_{L-1}^TA_{L}^T\nabla \ell\big(\mathcal{O}_L(\overline{\mathbf{w}}^L)\big)\\
\displaystyle B_L^T\nabla \ell\big(\mathcal{O}_L(\overline{\mathbf{w}}^L)\big)
\end{bmatrix}.
\end{align*}

We can keep on going until we get to
$$
\nabla f(\overline{\mathbf{w}}) = \begin{bmatrix}
\displaystyle 
B_1^T A_2^T A_3^T\dotsm A_{L-1}^TA_{L}^T\nabla \ell\big(\mathcal{O}_L(\overline{\mathbf{w}}^L)\big)\\
\displaystyle 
B_2^T A_3^T A_4^T\dotsm A_{L-1}^TA_{L}^T\nabla \ell\big(\mathcal{O}_L(\overline{\mathbf{w}}^L)\big)\\
\vdots\\
\displaystyle B_{L-1}^TA_{L}^T\nabla \ell\big(\mathcal{O}_L(\overline{\mathbf{w}}^L)\big)\\
\displaystyle B_L^T\nabla \ell\big(\mathcal{O}_L(\overline{\mathbf{w}}^L)\big)
\end{bmatrix} = \begin{bmatrix}
\displaystyle 
B_1^T A_2^T A_3^T\dotsm A_{L-1}^TA_{L}^T\\
\displaystyle 
B_2^T A_3^T A_4^T\dotsm A_{L-1}^TA_{L}^T\\
\vdots\\
\displaystyle B_{L-1}^TA_{L}^T\\
\displaystyle B_L^T
\end{bmatrix}\nabla \ell\big(\mathcal{O}_L(\overline{\mathbf{w}}^L)\big)
$$

### Algorithmic Computation

Recall that the matrices are constructed by computing the gradient of $\mathbf{g}_i$ at the location of the input at layer $i$. This starts at layer $1$ and then goes to layer $2$ and so one. But in order to compute the gradient we started with layer $L$ and then plug go backwards to layer $L-1$ until we get the formula above.

We can now turn this into an algorithm:

* **Initialize** Set $\mathbf{z}_1 = \mathbf{x}$.

* **For** Layer $i = 1,2,\dotsm, L$ (forward loop)

     * Define $\mathbf{z}_{i+1} = \mathbf{g}_i(\mathbf{z}_i,\mathbf{w}_i)$ as the output of layer $i$. Note that $\mathbf{z}_{L+1} = \mathcal{O}_L(\overline{\mathbf{w}}).$
     * Define $\begin{bmatrix} A_{i} & B_i\end{bmatrix} = J_{\mathbf{g}_i}(\mathbf{z}_i, \mathbf{w}_i)$ as the Jacobian.
* Define Loss:
    * Define $\mathbf{z}_{L+2} = \ell(\mathbf{z}_{L+1},\mathbf{y})$
    * Define $\mathbf{p}_{L+1} = \nabla \ell(\mathbf{z}_{L+1},\mathbf{y})$ where the gradient only includes the gradient w.r.t. the $z$ variables.
* **For** Layer $i = L,L-1,\dotsm, 1$ (backwards loop)
    * Define $\displaystyle 
\begin{bmatrix}    \mathbf{p}_{i}\\ \mathbf{q}_i \end{bmatrix} = \begin{bmatrix}
A_i^T\\B_i^T
\end{bmatrix}\mathbf{p}_{i+1}
    $
* **Output**
$\displaystyle \nabla f(\overline{\mathbf{w}}) = \begin{bmatrix}
\mathbf{q}_1\\\mathbf{q}_2\\\vdots\\\mathbf{q}_L
\end{bmatrix}\in \mathbb{R}^{r_1+\dotsm+r_L}.
$

## EXAMPLE

We pull up an advertising data set.

In [10]:
import pandas as pd
import numpy as np
from numpy import linalg as LA

We will use linear regression, so first we include some basic functions. 

The first ```mmids_gramschmidt``` is Gram-Schmidt which spits out the $QR$ decomposition of a matrix $A$. 

Then ```mmids_backsubs``` solves $U\mathbf{x}= \mathbf{b}$ where $U$ is an upper-triangular matrix using back-substitution. 

Finally, ```ls_by_qr``` solves least-squares by using the $QR$ decomposition which we know is
$$
\mathbf{x} = R^{-1} Q^T \mathbf{b}.
$$

In [3]:
def mmids_gramschmidt(A):
    B = np.copy(A)
    (n,m) = B.shape
    Q = np.zeros((n,m))
    R = np.zeros((m,m))
    for j in range(m):
        v = B[:,j]
        for i in range(j):
            R[i,j] = np.dot(Q[:,i], B[:,j])
            v -= R[i,j]*Q[:,i]$
            
        R[j,j] = LA.norm(v)
        Q[:,j] = v/R[j,j]
    return Q, R
def mmids_backsubs(U,b):
    m = b.shape[0]
    x = np.zeros(m)
    for i in reversed(range(m)):
        x[i] = (b[i] - np.dot(U[i,i+1:m],x[i+1:m]))/U[i,i]
    return x

def ls_by_qr(A, b):
    Q, R = mmids_gramschmidt(A)
    return mmids_backsubs(R, Q.T @ b)

Let's look at the first few rows of the data set.

In [4]:
df = pd.read_csv('CSV/advertising.csv')
df.head()

,Unnamed: 0,TV,radio,newspaper,sales
0,1,230.1,37.8,69.2,22.1
1,2,44.5,39.3,45.1,10.4
2,3,17.2,45.9,69.3,9.3
3,4,151.5,41.3,58.5,18.5
4,5,180.8,10.8,58.4,12.9


How big is the data set?

In [11]:
n = len(df.index)
print(n)

200


Let's get the TV advertising, radia advertising, and newspaper advertising as a vectors (they are lists here).

In [13]:
TV = df['TV'].to_numpy()
radio = df['radio'].to_numpy()
newspaper = df['newspaper'].to_numpy()
sales = df['sales'].to_numpy()
print(TV[0:5])

[230.1  44.5  17.2 151.5 180.8]


Let's now form the matrix $A$ whose columns are the TV, radio and newspaper spending and there is an additional column of all ones. We'll print just the first few rows of $A$.

In [17]:
features = np.stack((TV, radio, newspaper), axis=-1)
A = np.c_[np.ones(n), features]
print(A[0:5,:])

[[  1.  230.1  37.8  69.2]
 [  1.   44.5  39.3  45.1]
 [  1.   17.2  45.9  69.3]
 [  1.  151.5  41.3  58.5]
 [  1.  180.8  10.8  58.4]]


Let's now look at the least squares coefficients.

In [20]:
coeff = ls_by_qr(A, sales)
print(coeff)

[ 2.93888937e+00  4.57646455e-02  1.88530017e-01 -1.03749304e-03]


This means that 
$$
\text{SALES} \approx 2.94 + 0.05\times \text{TV} + 0.19 \times\text{RADIO} + 0.00\times\text{PRINT}.
$$

How good is this?

In [21]:
np.mean((A @ coeff - sales)**2)

2.7841263145109365

What happens if we use a shallow (i.e. small $L$) multi-layer function?

In [9]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [22]:
train_dataset = tf.data.Dataset.from_tensor_slices((features, sales))


In [23]:
BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = 100
train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)

In [24]:
model = tf.keras.Sequential([
 layers.Dense(input_dim=3, units=1)
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1)                 4         
                                                                 
Total params: 4 (16.00 Byte)
Trainable params: 4 (16.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [25]:
model.compile(
 optimizer=tf.optimizers.SGD(learning_rate=1e-5),
 loss='mean_squared_error'
 )


In [26]:
model.fit(
 train_dataset, batch_size=64, epochs=np.int32(1e4), verbose=0
 )

In [27]:
print(model.layers[0].get_weights())

[array([[0.0548817 ],
       [0.21836095],
       [0.01579611]], dtype=float32), array([0.30926073], dtype=float32)]


In [28]:
model.evaluate(train_dataset)


4/4 [==============================] - 0s 3ms/step - loss: 3.9137


3.9137401580810547